In [16]:
import random
import numpy as np
import pandas as pd
import duckdb

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

Autre exemple: assurance santé. <br />
On pourrait vouloir grouper:
- par type de soin
- par type de contrat
- par groupe d'age du bénéficiaire
- et par toutes les combinaisons possibles des groupes ci-dessus:
- par type de soin et groupe d'age
- par type de contrat et groupe d'age
- par type de soin et type de contrat
- par type de soin, contrat, et groupe d'âge

# Data

In [2]:
random.seed(42)
num_samples = 1000

contrats = ["senior", "jeunes", "expat", "famille", "salarié"]
sexe = ["homme", "femme"]
type_acte = {"pharmacie": 15,
"consultation_generaliste": 25,
"hospitalisation": 2800,
"biologie": 150,
"radio": 1300,
"maternite": 1700}
groupe_age = ["18-25", "25-45", "45-65", "65+"]
annee = [2017, 2018, 2019]


# Initialize empty lists to store the data
contrats_data = []
sexe_data = []
type_acte_data = []
groupe_age_data = []
annee_data = []
cost_data = []

# Generate random data for each category
for _ in range(num_samples):
    contrats_data.append(random.choice(contrats))
    sexe_data.append(random.choice(sexe))
    if sexe_data == "femme":
        type_acte_choice = random.choice(list(type_acte.keys()))
    else:
        type_acte_options = list(type_acte.keys())
        type_acte_options.remove("maternite")
        type_acte_choice = random.choice(type_acte_options)
        
    type_acte_data.append(type_acte_choice)
    cost_mean = type_acte[type_acte_choice]
    cost_data.append(np.random.normal(cost_mean, cost_mean // 3.5))  # Assuming a standard deviation of 50 for costs
    groupe_age_data.append(random.choice(groupe_age))
    annee_data.append(random.choice(annee))


In [3]:
# Create a DataFrame to store the dataset
df = pd.DataFrame({
    'type_contrat': contrats_data,
    'sexe': sexe_data,
    'type_acte': type_acte_data,
    'groupe_age': groupe_age_data,
    'annee': annee_data,
    'montant_rembourse': cost_data
})
df

,type_contrat,sexe,type_acte,groupe_age,annee,montant_rembourse
0,senior,homme,hospitalisation,25-45,2017,2697.306310
1,jeunes,homme,radio,18-25,2019,1843.308677
2,famille,homme,pharmacie,18-25,2017,8.023486
3,jeunes,homme,radio,25-45,2019,1593.518591
4,salarié,femme,consultation_generaliste,65+,2019,23.891086
...,...,...,...,...,...,...
995,famille,homme,biologie,45-65,2018,97.176503
996,jeunes,homme,hospitalisation,65+,2019,3250.830132
997,salarié,femme,pharmacie,18-25,2017,15.289353
998,expat,homme,hospitalisation,45-65,2019,1779.672811


# Exercices: Rollup

Voici le montant global remboursé par la caisse d'assurance sur la période 2017-2019

In [4]:
query = """
SELECT SUM(montant_rembourse) 
FROM df
"""
duckdb.sql(query)

┌────────────────────────┐
│ sum(montant_rembourse) │
│         double         │
├────────────────────────┤
│      859578.9569530528 │
└────────────────────────┘

Votre PO vous demande le montant total remboursé par type de contrat

In [5]:
query = """
SELECT
    type_contrat,
    sum(montant_rembourse)
FROM df
GROUP BY type_contrat


"""
duckdb.sql(query).df()

,type_contrat,sum(montant_rembourse)
0,salarié,191312.997347
1,expat,143060.843722
2,famille,179391.089487
3,jeunes,173000.871909
4,senior,172813.154488


In [6]:
# %load solutions/7groupby_type_contrat.py
query = """
SELECT type_contrat, SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat
"""
duckdb.sql(query)


┌──────────────┬────────────────────────┐
│ type_contrat │ sum(montant_rembourse) │
│   varchar    │         double         │
├──────────────┼────────────────────────┤
│ salarié      │     191312.99734668847 │
│ jeunes       │     173000.87190925682 │
│ expat        │      143060.8437216833 │
│ senior       │     172813.15448832474 │
│ famille      │     179391.08948709862 │
└──────────────┴────────────────────────┘

Votre Manager vous transmet une autre demande du PO: <br />
il vous demande le montant total remboursé par type de contrat
ET par type d'acte

In [9]:
query = """
SELECT
    type_contrat,
    type_acte,
    sum(montant_rembourse)
FROM df
GROUP BY type_contrat, type_acte
ORDER BY type_contrat, type_acte

"""
duckdb.sql(query).df()

,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6758.035524
1,expat,consultation_generaliste,1188.059457
2,expat,hospitalisation,82289.578360
3,expat,pharmacie,482.947477
4,expat,radio,52342.222903
5,famille,biologie,8025.711693
6,famille,consultation_generaliste,770.576638
7,famille,hospitalisation,110061.879264
8,famille,pharmacie,438.828990
9,famille,radio,60094.092903


In [8]:
# %load solutions/8groupby_typecontrat_typeacte.py

Le PO revient en râlant: le manager n'avait rien compris ! <br />
Il veut le montant total remboursé par type de contrat <br />
ET le montant total remboursé par type d'acte

Hint: commencez simple, faites le avec un Union

In [10]:
# %load solutions/9groupby_unions.py
query = """
SELECT type_contrat AS typologie,
SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat

UNION

SELECT type_acte AS typologie, 
SUM(montant_rembourse) 
FROM df
GROUP BY type_acte
"""
duckdb.sql(query)


┌──────────────────────────┬────────────────────────┐
│        typologie         │ sum(montant_rembourse) │
│         varchar          │         double         │
├──────────────────────────┼────────────────────────┤
│ expat                    │      143060.8437216833 │
│ hospitalisation          │      551729.4269225589 │
│ salarié                  │     191312.99734668847 │
│ radio                    │      267799.7509364036 │
│ senior                   │     172813.15448832474 │
│ consultation_generaliste │      5030.343557175703 │
│ biologie                 │     32484.843662744184 │
│ jeunes                   │     173000.87190925682 │
│ pharmacie                │     2534.5918741694336 │
│ famille                  │     179391.08948709862 │
├──────────────────────────┴────────────────────────┤
│ 10 rows                                 2 columns │
└───────────────────────────────────────────────────┘

Votre tech lead vient vous voir:

<blockquote> Ton code fait le taff, mais j'ai récemment lu un article sur les GROUPING SETS, je pense que ça permettrait de simplifier le code sur ton problème, tu peux implémenter ça ? Merci ! </blockquote>

In [11]:
# %load solutions/10groupby_groupingsets.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY GROUPING SETS
(type_contrat, type_acte)
-- Notez la difference avec ((type_contrat, type_acte))
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query)


┌──────────────┬──────────────────────────┬────────────────────────┐
│ type_contrat │        type_acte         │ sum(montant_rembourse) │
│   varchar    │         varchar          │         double         │
├──────────────┼──────────────────────────┼────────────────────────┤
│ expat        │ NULL                     │      143060.8437216833 │
│ famille      │ NULL                     │     179391.08948709862 │
│ jeunes       │ NULL                     │     173000.87190925682 │
│ salarié      │ NULL                     │     191312.99734668847 │
│ senior       │ NULL                     │     172813.15448832474 │
│ NULL         │ biologie                 │     32484.843662744184 │
│ NULL         │ consultation_generaliste │      5030.343557175703 │
│ NULL         │ hospitalisation          │      551729.4269225589 │
│ NULL         │ pharmacie                │     2534.5918741694336 │
│ NULL         │ radio                    │      267799.7509364036 │
├──────────────┴──────────────────

Votre manager revient à la charge:
<blockquote> Perso, je trouve que c'était très bien d'avoir aussi le montant total remboursé par type de contrat
ET par type d'acte. J'ai discuté avec le PO et il est OK. Garde ce que t'as fait, mais remets aussi les subdivisions </blockquote>

Vous décidez d'utiliser ROLLUP pour obtenir tout ça facilement:

In [12]:
# %load solutions/11rollup.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY ROLLUP
(type_contrat, type_acte)
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query).df()


,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6758.035524
1,expat,consultation_generaliste,1188.059457
2,expat,hospitalisation,82289.578360
3,expat,pharmacie,482.947477
4,expat,radio,52342.222903
5,expat,None,143060.843722
6,famille,biologie,8025.711693
7,famille,consultation_generaliste,770.576638
8,famille,hospitalisation,110061.879264
9,famille,pharmacie,438.828990


# Exercices: CUBE

Votre tech lead vérifie votre code et vous dit:

<blockquote> Tu t'es planté. ROLLUP ça enlève progressivement le niveau de regroupement le plus à droite de ta liste. Résultat: on n'a pas les sommes par type_acte uniquement (il manque 5 lignes) 
Il faut trouver une autre solution ! </blockquote>


Il nous faut donc:
- la somme par type d'acte (5 lignes),
- la somme par type de contrat (5 lignes),
- ET la somme par type d'acte + type de contrat (25 lignes),

Et effectivement, avec ROLLUP il nous manquait la somme par type d'acte uniquement (5 lignes)

Vous décidez d'utiliser un CUBE Pour avoir toutes les options:

In [13]:
# %load solutions/12cube.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY CUBE
(type_contrat, type_acte)
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query).df()


,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6758.035524
1,expat,consultation_generaliste,1188.059457
2,expat,hospitalisation,82289.578360
3,expat,pharmacie,482.947477
4,expat,radio,52342.222903
5,expat,None,143060.843722
6,famille,biologie,8025.711693
7,famille,consultation_generaliste,770.576638
8,famille,hospitalisation,110061.879264
9,famille,pharmacie,438.828990


# Grand' Final

En lisant le rapport fourni par votre P.O. sur la base de vos chiffres <br />
Votre N+2 a eu plein d'idées.

<img src="images/asterix_plein_idees.gif" />
<p style="text-align:center"><i>Le N+2</i></p>

<blockquote> Je veux la somme des montants remboursés par:
 type_contrat / type_acte / groupe_age / sexe / annee
</blockquote>

et je veux avoir les chiffres globaux pour chacune de ces sous-catégories:
- type_contrat / type_acte / groupe_age / sexe /
- type_contrat / type_acte / groupe_age /
- type_contrat / type_acte / 
- type_contrat / 

Maintenant que vous connaissez ROLLUP, ça devrait être un jeu d'enfant:

In [17]:
# %load solutions/13rollup_finale.py
query = """
SELECT type_contrat, type_acte, groupe_age, sexe, annee,
SUM(montant_rembourse) 
FROM df
GROUP BY ROLLUP
( type_contrat, type_acte, groupe_age, sexe, annee )
ORDER BY type_contrat, type_acte, groupe_age, sexe, annee 
"""
duckdb.sql(query).df()


,type_contrat,type_acte,groupe_age,sexe,annee,sum(montant_rembourse)
0,expat,biologie,18-25,femme,2017.0,206.098218
1,expat,biologie,18-25,femme,2018.0,156.389220
2,expat,biologie,18-25,femme,2019.0,155.849199
3,expat,biologie,18-25,femme,NaN,518.336637
4,expat,biologie,18-25,homme,2017.0,143.708954
5,expat,biologie,18-25,homme,2018.0,133.825611
6,expat,biologie,18-25,homme,2019.0,492.053184
7,expat,biologie,18-25,homme,NaN,769.587748
8,expat,biologie,18-25,None,NaN,1287.924386
9,expat,biologie,25-45,femme,2017.0,249.881862
